In [2]:
import os
import traceback
import pandas as pd
import geopandas as gpd
from geopandas.tools import sjoin
import sqlalchemy
import matplotlib.pyplot as plt
import numpy as np

user='hifld_summer'
pw='hifld_summer'
name='hifld_summer'
host='moria'
port=2023
engine = sqlalchemy.create_engine(
    f'postgresql://{user}:{pw}@{host}:{port}/{name}'
    )

In [23]:
sql_p="""
select geom, parcel_lid, use_code_std_lps from parcels.alabama limit 5000
"""

gdf_p = gpd.GeoDataFrame.from_postgis(sql_p, con=engine)

Index(['geom', 'parcel_lid', 'use_code_std_lps'], dtype='object')
epsg:4326
mh_classifier
0    4833
1     167
Name: count, dtype: int64


In [83]:
sql_f="""
select * from results.all_features limit 5000
"""

gdf_f = gpd.GeoDataFrame.from_postgis(sql_f, con=engine, crs='EPSG:4326')
gdf_f.columns


Index(['geom', 'build_id', 'prim_occ', 'new_sqmeters', 'perimeter',
       'vertex_count', 'length', 'width', 'long_side', 'short_side',
       'ls_ratio', 'pp_compactness', 'min_bounding_circle_area',
       'reock_compactness', 'sch_compactness', 'lon', 'lat',
       'points_in_polygon_buffers', 'ls_ratios_mean', 'ls_ratios_std',
       'ls_ratios_min', 'ls_ratios_max', 'area_mean', 'area_std', 'area_min',
       'area_max'],
      dtype='object')

In [75]:
gdf_p = gpd.read_file('C:\\Users\\zsn\\Desktop\\results\\src\\modeling\\shapefiles\\parcel_sample.shp')
gdf_p = gdf_p.drop(columns=['use_code_m', 'use_code_1', 'use_code_2'])
gdf_p.columns

Index(['parcel_lid', 'use_code_s', 'geometry'], dtype='object')

In [64]:
gdf_f = gpd.read_file('C:\\Users\\zsn\\Desktop\\results\\src\\modeling\\shapefiles\\all_features_sample.shp')

<Projected CRS: ESRI: 102003>
Name: USA_Contiguous_Albers_Equal_Area_Conic
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: United States (USA) - CONUS onshore - Alabama; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming.
- bounds: (-124.79, 24.41, -66.91, 49.38)
Coordinate Operation:
- name: USA_Contiguous_Albers_Equal_Area_Conic
- method: Albers Equal Area
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [82]:
print(gdf_f.crs)
gdf_p = gdf_p.to_crs('epsg: 4326')
print(gdf_p.crs)


joined = gpd.sjoin(gdf_f, gdf_p, how='left', predicate='intersects')

joined["mh_classifier"] = (joined["use_code_s"] == '1006').astype(int)
print(joined.head())
print(joined.mh_classifier.value_counts())


# joined.to_file('C:\\Users\\zsn\\Desktop\\results\\src\\modeling\\shapefiles\\SJOINTEST')

epsg: 4326
epsg: 4326
    build_id                prim_occ  new_sqmete  perimeter  vertex_cou  \
0  1663812.0       Manufactured Home  119.512368  47.107048         4.0   
1  1663846.0  Single Family Dwelling  122.575987  44.543347         4.0   
2  1663956.0  Single Family Dwelling  275.796851  66.817431         4.0   
3  1663962.0  Single Family Dwelling  165.476946  59.544011         4.0   
4  1663977.0       Manufactured Home  122.576894  44.550458         4.0   

      length      width  long_side  short_side  ls_ratio  ...  ls_ratio_3  \
0  17.636297  11.791490  17.636297   11.791490  1.495680  ...    2.317781   
1  13.651776  14.997799  14.997799   13.651776  1.098597  ...    2.635884   
2  21.225885  22.925539  22.925539   21.225885  1.080075  ...    2.635884   
3  23.090324   9.962253  23.090324    9.962253  2.317781  ...    2.635884   
4  15.188981  15.733438  15.733438   15.188981  1.035846  ...    2.317781   

    area_mean   area_std   area_min    area_max  \
0  162.822132